In [1]:
from Executor import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("data_aug").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [14]:
executor.load_model_from_file("weights.first.h5")
executor.compile(learn_rate=0.001)

for layer in executor.vgg.model.layers:
    layer.trainable=False
executor.vgg.model.layers[-1].trainable=True

data_aug_generator = image.ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
          shear_range=0.15, zoom_range=0.1, channel_shift_range=10, horizontal_flip=True, vertical_flip=True)

executor.train_batches = executor.vgg.get_batches(executor.train_path, gen=data_aug_generator, batch_size=executor.batch_size)

# gives a pretty decent result, with val_loss ending at 0.70, which is around what 'weights.frist.h5' settled at
executor.train_for_epochs(2)

Found 6255 images belonging to 3 classes.
Epoch 1/2
6255/6255 [==============================] - 205s - loss: 1.0672 - acc: 0.5768 - val_loss: 0.7112 - val_acc: 0.6934
Epoch 2/2
6255/6255 [==============================] - 205s - loss: 1.0127 - acc: 0.5661 - val_loss: 0.7151 - val_acc: 0.6856


<Executor.Executor instance at 0x7f5778366320>

In [15]:
executor.train_for_epochs(2)

Epoch 1/2
6255/6255 [==============================] - 205s - loss: 1.0153 - acc: 0.5583 - val_loss: 0.7421 - val_acc: 0.6553
Epoch 2/2
6255/6255 [==============================] - 205s - loss: 1.0205 - acc: 0.5536 - val_loss: 0.7501 - val_acc: 0.6662


<Executor.Executor instance at 0x7f5778366320>

In [17]:
executor.make_linear_layers_trainable()
executor.compile(learn_rate=0.00001)

('first dense layer at index: ', 33)
all dense layers set trainable.


<Executor.Executor instance at 0x7f5778366320>

In [18]:
executor.train_for_epochs(3)

Epoch 1/3
6255/6255 [==============================] - 211s - loss: 11.1814 - acc: 0.2978 - val_loss: 10.9674 - val_acc: 0.3196
Epoch 2/3
6255/6255 [==============================] - 210s - loss: 11.1551 - acc: 0.3079 - val_loss: 11.0610 - val_acc: 0.3138
Epoch 3/3
6255/6255 [==============================] - 211s - loss: 11.3226 - acc: 0.2975 - val_loss: 11.1963 - val_acc: 0.3054


<Executor.Executor instance at 0x7f5778366320>